# Projet :   Analyse de données <span style='color:Blue'> Twitter  </span> : Python pour la data-science 🐍🗃️

#### Auteurs : Ambroise Bouru, Juliette Veillon, Florent Le Clerc

Ce projet a été effectué dans le cadre de l'unité d'enseignement "Python pour la data-science" dispensé à l'ENSAE. Conformément aux attendus de cet enseignement, ce projet est divisé en trois parties : une partie "récupération et traitement des données", une partie "modélisation", ainsi qu'une partie "analyse descriptive et représentation graphique".

<br /><br />

## Problématique

Dans ce projet, nous tenterons de répondre à la problématique suivante : **Dans quelle mesure l'analyse de cluster de graphes permet-elle de mettre en évidence des caractéristiques propres à différents groupes d'utilisateurs de Twitter ?**

Afin de répondre à cette problématique, nous avons utilisé un jeu de données volontairement très spécifique, à savoir l'ensemble des utilisateurs français ayant tweeté sur le match 🇦🇷 - 🇭🇷 ayant eu lieu le mardi 13 décembre.

Le jeu de données obtenus, contenant environ 9000 tweets, nous a permis de réaliser une modélisation pertinente sans que cela ne pose de problème en terme de temps d'exécution.

<br />

### I. Récupération et traitement des données

La récupération des données s'est faite à l'aide de l'API 2.0, mise à disposition gratuitement par Twitter (https://developer.twitter.com/en/docs/api-reference-index).

La manipulation des requêtes de cette API s'est faite via le module "twitter" de Python.

Dans le dossier correspondant sur GitHub, nous avons scindé cette partie en deux notebooks. Le premier notebook contient les requêtes ayant permis d'accéder au jeu de données des utilisateurs français ayant tweeté sur le match. Le second contient, quant à lui, les requêtes ayant permis de construire la base de données de tous les amis des utilisateurs obtenus dans le premier notebook.

<br />

### II. Modélisation

La base de données des relations créée nous permet ensuite de construire le graphe des relations.

Dans le dossier correspondant, nous avons fait deux choses principalement. Tout d'abord, nous avons construit le graphe des relations à partir du jeu de données obtenus. Ensuite, nous avons réalisé le clustering de ce graphe à l'aide de trois algorithmes différents.

Les résultats obtenus nous ont permis de conclure que l'algorithme le plus pertinent dans le cadre de notre projet est l'algorithme de Louvain. En effet, le nombre de clusters obtenus nous a semblé ni trop élévé ni trop faible. À l'inverse, l'algorithme de clustering par propagation d'étiquette semblait 

<br />

### III. Analyse des données
